# Softmax RNN

In [5]:
# SoftMax Character Sequence RNN
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn


In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
           "collect wood and don't assign them tasks and work, but rather "
           "teach them to long for the endless immensity of the sea.")

In [18]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10


dataX = []
dataY = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1: i+seq_length + 1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [19]:
# Stacked RNN
tf.reset_default_graph() # 변수가 이미 생성되어 있을 때 사용.

X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)

# Mack a LSTM Cell with hidden size
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs : unfolding size X hidden_size, state = hidden_size
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [21]:
# SoftMax
#outputs = tf.reshape(outputs, batch_size, seq_length, num_classes)
x_for_softmax = tf.reshape(outputs, [-1, hidden_size])

softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(x_for_softmax, softmax_w) + softmax_b

outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])

In [22]:
# all weights are 1
weights = tf.ones([batch_size, seq_length])

seq_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y,
                                           weights=weights)
mean_loss = tf.reduce_mean(seq_loss)

train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)

# Training and Print results

In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
    
for i in range(500):
    _, l, results = sess.run([train_op, mean_loss, outputs],
                            feed_dict={X:dataX, Y:dataY})
        
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

0 0            3.1799884
0 1            3.1799884
0 2            3.1799884
0 3            3.1799884
0 4            3.1799884
0 5            3.1799884
0 6            3.1799884
0 7            3.1799884
0 8            3.1799884
0 9            3.1799884
0 10            3.1799884
0 11            3.1799884
0 12            3.1799884
0 13            3.1799884
0 14            3.1799884
0 15            3.1799884
0 16            3.1799884
0 17            3.1799884
0 18            3.1799884
0 19            3.1799884
0 20            3.1799884
0 21            3.1799884
0 22            3.1799884
0 23            3.1799884
0 24            3.1799884
0 25            3.1799884
0 26            3.1799884
0 27            3.1799884
0 28            3.1799884
0 29            3.1799884
0 30            3.1799884
0 31            3.1799884
0 32            3.1799884
0 33            3.1799884
0 34            3.1799884
0 35            3.1799884
0 36            3.1799884
0 37            3.1799884
0 38            3.1799

3 111            2.9229958
3 112            2.9229958
3 113         tt 2.9229958
3 114       tttt 2.9229958
3 115       tttt 2.9229958
3 116       tttt 2.9229958
3 117      ttttt 2.9229958
3 118       tttt 2.9229958
3 119        ttt 2.9229958
3 120       tttt 2.9229958
3 121       tttt 2.9229958
3 122      ttttt 2.9229958
3 123        ttt 2.9229958
3 124         t  2.9229958
3 125            2.9229958
3 126       tttt 2.9229958
3 127      ttttt 2.9229958
3 128       tttt 2.9229958
3 129            2.9229958
3 130            2.9229958
3 131            2.9229958
3 132            2.9229958
3 133            2.9229958
3 134            2.9229958
3 135            2.9229958
3 136            2.9229958
3 137            2.9229958
3 138            2.9229958
3 139            2.9229958
3 140            2.9229958
3 141            2.9229958
3 142            2.9229958
3 143       tttt 2.9229958
3 144       tttt 2.9229958
3 145      ttttt 2.9229958
3 146            2.9229958
3 147          t 2.9229958
3

6 3  ttttttttt 2.9030528
6 4  ttttttttt 2.9030528
6 5  ttttttttt 2.9030528
6 6 ottttttttt 2.9030528
6 7  ttttttttt 2.9030528
6 8 ottttttttt 2.9030528
6 9  ttttttttt 2.9030528
6 10 ootttttttt 2.9030528
6 11 ootttttttt 2.9030528
6 12 ottttttttt 2.9030528
6 13  ttttttttt 2.9030528
6 14 ottttttttt 2.9030528
6 15  ttttttttt 2.9030528
6 16  ttttttttt 2.9030528
6 17   tttttttt 2.9030528
6 18  ttttttttt 2.9030528
6 19 ootttttttt 2.9030528
6 20 ootttttttt 2.9030528
6 21 ottttttttt 2.9030528
6 22 ottttttttt 2.9030528
6 23  ttttttttt 2.9030528
6 24 tttttttttt 2.9030528
6 25   tttttttt 2.9030528
6 26  ttttttttt 2.9030528
6 27 ottttttttt 2.9030528
6 28 ootttttttt 2.9030528
6 29 ottttttttt 2.9030528
6 30  ttttttttt 2.9030528
6 31  ttttttttt 2.9030528
6 32  ttttttttt 2.9030528
6 33 ootttttttt 2.9030528
6 34 ootttttttt 2.9030528
6 35 ottttttttt 2.9030528
6 36  ttttttttt 2.9030528
6 37  ttttttttt 2.9030528
6 38  ttttttttt 2.9030528
6 39 ottttttttt 2.9030528
6 40  ttttttttt 2.9030528
6 41  ttttttttt 2.9

8 53            2.852373
8 54            2.852373
8 55            2.852373
8 56            2.852373
8 57            2.852373
8 58            2.852373
8 59            2.852373
8 60            2.852373
8 61            2.852373
8 62            2.852373
8 63            2.852373
8 64            2.852373
8 65            2.852373
8 66            2.852373
8 67            2.852373
8 68            2.852373
8 69            2.852373
8 70            2.852373
8 71            2.852373
8 72            2.852373
8 73            2.852373
8 74            2.852373
8 75            2.852373
8 76            2.852373
8 77            2.852373
8 78            2.852373
8 79            2.852373
8 80            2.852373
8 81            2.852373
8 82            2.852373
8 83            2.852373
8 84            2.852373
8 85            2.852373
8 86            2.852373
8 87            2.852373
8 88            2.852373
8 89            2.852373
8 90            2.852373
8 91            2.852373
8 92            2.852373


10 144            2.8433886
10 145            2.8433886
10 146            2.8433886
10 147            2.8433886
10 148            2.8433886
10 149            2.8433886
10 150            2.8433886
10 151            2.8433886
10 152            2.8433886
10 153            2.8433886
10 154            2.8433886
10 155            2.8433886
10 156            2.8433886
10 157            2.8433886
10 158            2.8433886
10 159            2.8433886
10 160            2.8433886
10 161            2.8433886
10 162            2.8433886
10 163            2.8433886
10 164            2.8433886
10 165            2.8433886
10 166            2.8433886
10 167            2.8433886
11 0            2.8318238
11 1            2.8318238
11 2            2.8318238
11 3            2.8318238
11 4            2.8318238
11 5            2.8318238
11 6            2.8318238
11 7            2.8318238
11 8            2.8318238
11 9            2.8318238
11 10            2.8318238
11 11            2.8318238
11 12         

...

490 123 nch them t 0.22631791
490 124 nh them to 0.22631791
490 125 t them to  0.22631791
490 126 ethem to l 0.22631791
490 127 toem ta lo 0.22631791
490 128 her ta lon 0.22631791
490 129 er ta long 0.22631791
490 130 n ta long  0.22631791
490 131  ta long f 0.22631791
490 132 to bong fo 0.22631791
490 133 h bong for 0.22631791
490 134 nlong for  0.22631791
490 135 tong for t 0.22631791
490 136 eng for th 0.22631791
490 137 n' for the 0.22631791
490 138 d for the  0.22631791
490 139  for the e 0.22631791
490 140 tor the en 0.22631791
490 141 or the end 0.22631791
490 142 n the endl 0.22631791
490 143  the endle 0.22631791
490 144 toemendles 0.22631791
490 145 herendless 0.22631791
490 146 erendless  0.22631791
490 147 ntndless i 0.22631791
490 148 tndless im 0.22631791
490 149 nsless imm 0.22631791
490 150 d ess imme 0.22631791
490 151  ess immen 0.22631791
490 152 e s immens 0.22631791
490 153 ns immensi 0.22631791
490 154 iiimmensit 0.22631791
490 155 iimmensity 0.22631791
490 156 tm

492 131  ta long f 0.22634159
492 132 to long fo 0.22634159
492 133 h bong for 0.22634159
492 134 nlong for  0.22634159
492 135 tong for t 0.22634159
492 136 eng for th 0.22634159
492 137 n' for the 0.22634159
492 138 d for the  0.22634159
492 139  for the e 0.22634159
492 140 tor the en 0.22634159
492 141  r the end 0.22634159
492 142 n the endl 0.22634159
492 143  the endle 0.22634159
492 144 toemendles 0.22634159
492 145 herendless 0.22634159
492 146 erendless  0.22634159
492 147 rtndless i 0.22634159
492 148 tndless im 0.22634159
492 149 rsless imm 0.22634159
492 150 d ess imme 0.22634159
492 151  ess immen 0.22634159
492 152 e s immens 0.22634159
492 153 rs immensi 0.22634159
492 154 iiimmensit 0.22634159
492 155 iimmensity 0.22634159
492 156 tmmensity  0.22634159
492 157 tmensity o 0.22634159
492 158  ensity of 0.22634159
492 159  nsity of  0.22634159
492 160 rsity of t 0.22634159
492 161 dity of th 0.22634159
492 162 igy of the 0.22634159
492 163 ty of the  0.22634159
492 164 h 

495 0 f you want 0.22646411
495 1 oyou want  0.22646411
495 2 tou want t 0.22646411
495 3 ou want to 0.22646411
495 4 n want to  0.22646411
495 5 mwant to b 0.22646411
495 6 tont to bu 0.22646411
495 7 ont to bui 0.22646411
495 8 nd to buil 0.22646411
495 9 d to build 0.22646411
495 10  ro build  0.22646411
495 11 to build a 0.22646411
495 12   build a  0.22646411
495 13 nbuild a s 0.22646411
495 14 tutld a sh 0.22646411
495 15 utld a shi 0.22646411
495 16 mld a ship 0.22646411
495 17 fd a ship, 0.22646411
495 18 e a ship,  0.22646411
495 19  a ship, d 0.22646411
495 20 tnship, do 0.22646411
495 21 nship, don 0.22646411
495 22 thip, don' 0.22646411
495 23  ip, don't 0.22646411
495 24 ep, don't  0.22646411
495 25 f, don't d 0.22646411
495 26 e don't dr 0.22646411
495 27  don't dru 0.22646411
495 28 ton't drum 0.22646411
495 29  n't drum  0.22646411
495 30 n't drum u 0.22646411
495 31 dt drum up 0.22646411
495 32 t drum up  0.22646411
495 33  rrum up p 0.22646411
495 34 toum up pe 0.2264

497 91  em tosks  0.22665319
497 92 em tosks a 0.22665319
497 93 m tosks an 0.22665319
497 94  tosks and 0.22665319
497 95 tosks and  0.22665319
497 96  sks and w 0.22665319
497 97 nks and wo 0.22665319
497 98  s and wor 0.22665319
497 99 , and work 0.22665319
497 100  and work, 0.22665319
497 101 tnd work,  0.22665319
497 102 nd work, b 0.22665319
497 103 d work, bu 0.22665319
497 104  aork, but 0.22665319
497 105 took, but  0.22665319
497 106 ook, but r 0.22665319
497 107 nk, but ra 0.22665319
497 108  , but rat 0.22665319
497 109 , but rath 0.22665319
497 110  dut rathe 0.22665319
497 111 tut rather 0.22665319
497 112 ut rathert 0.22665319
497 113 m ratherte 0.22665319
497 114  rathertea 0.22665319
497 115 tatherteac 0.22665319
497 116  therteach 0.22665319
497 117 nherteach  0.22665319
497 118  emteach t 0.22665319
497 119 em each th 0.22665319
497 120 mteach the 0.22665319
497 121  each them 0.22665319
497 122  ach them  0.22665319
497 123 mch them t 0.22665319
497 124 nh them to 

499 107 nk, but ra 0.22685303
499 108  , but rat 0.22685303
499 109 , but rath 0.22685303
499 110  but rathe 0.22685303
499 111 tut rather 0.22685303
499 112 ut rathert 0.22685303
499 113 m ratherte 0.22685303
499 114  rathertea 0.22685303
499 115 tatherteac 0.22685303
499 116  therteach 0.22685303
499 117 nherteach  0.22685303
499 118  emteach t 0.22685303
499 119 emteach th 0.22685303
499 120 mteach the 0.22685303
499 121  each them 0.22685303
499 122  ach them  0.22685303
499 123 mch them t 0.22685303
499 124 nh them to 0.22685303
499 125 o them to  0.22685303
499 126 ethem to l 0.22685303
499 127 toem to lo 0.22685303
499 128  em to lon 0.22685303
499 129 em to long 0.22685303
499 130 m to long  0.22685303
499 131  to long f 0.22685303
499 132 to bong fo 0.22685303
499 133   bong for 0.22685303
499 134 nbong for  0.22685303
499 135 tong for t 0.22685303
499 136 eng for th 0.22685303
499 137 n' for the 0.22685303
499 138 d for the  0.22685303
499 139 efor the e 0.22685303
499 140 to

In [24]:
# Let's print the Last char of each result to check it works
results = sess.run(outputs, feed_dict={X:dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0: # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

m you want to build a ship, don't drum up people together tocollect wood and don't assign them tasks and work, but ratherteach them to long for the endless immensity of the sea.